In [1]:
import pulp

problem = pulp.LpProblem('SLE', pulp.LpMaximize)

x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

problem += 120 * x + 150 * y == 1440
problem += x + y == 10

status = problem.solve()

print('Status: ', pulp.LpStatus[status])
print('x=', x.value(), 'y=', y.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/isobekosuke/home/operations_research/venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/e65f0df4d3484ca082f8512d785ca997-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/e65f0df4d3484ca082f8512d785ca997-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 13 RHS
At line 16 BOUNDS
At line 20 ENDATA
Problem MODEL has 2 rows, 3 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-2) rows, 0 (-3) columns and 0 (-4) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00
Option for 

In [2]:
import pulp

problem = pulp.LpProblem('LP', pulp.LpMaximize)

x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0
problem += x + 2 * y

status = problem.solve()

print('Status:', pulp.LpStatus[status])
print('x=', x.value(), 'y=', y.value(), 'obj=', problem.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/isobekosuke/home/operations_research/venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/0abccd26b55b4b58aebe304a8e5ffefe-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/0abccd26b55b4b58aebe304a8e5ffefe-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 18 RHS
At line 23 BOUNDS
At line 26 ENDATA
Problem MODEL has 4 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
0  Obj -0 Dual inf 2.999998 (2)
2  Obj 26
Optimal - objective value 26
After Postsolve, objective 26, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 26 - 2 iterations time 0.002, Presolve 0.00
Option for printi

## 2.3 規模の大きな数理最適化問題を Python の数理最適化ライブラリで解く

In [17]:
import pandas as pd
import pulp

In [18]:
stock_df = pd.read_csv('stocks.csv')
stock_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [19]:
require_df = pd.read_csv('requires.csv')
require_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


In [20]:
gain_df = pd.read_csv('gains.csv')
gain_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [40]:
P = gain_df['p'].tolist()
print(P)

['p1', 'p2', 'p3', 'p4']


In [41]:
M = stock_df['m'].tolist()
print(M)

['m1', 'm2', 'm3']


### ③ 定数の定義

In [42]:
stock = {row.m:row.stock for row in stock_df.itertuples()}
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [43]:
require = {(row.p, row.m):row.require for row in require_df.itertuples()}
print(require)

{('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3, ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2, ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}


In [44]:
gain = {row.p:row.gain for row in gain_df.itertuples()}
print(gain)

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}


### ④ 線形計画問題の定義

In [45]:
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

### ⑤ 変数の定義

In [46]:
x = pulp.LpVariable.dicts('x', P, cat='Continuous')
print(x)

{'p1': x_p1, 'p2': x_p2, 'p3': x_p3, 'p4': x_p4}


### ⑥ 制約式の定義

In [47]:
for p in P:
    problem += x[p] >= 0

In [48]:
for m in M:
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]

### ⑦ 目的関数の定義

In [49]:
problem += pulp.lpSum([gain[p] * x[p] for p in P])

In [50]:
status = problem.solve()
print('Status:', pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/isobekosuke/home/operations_research/venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/9e4a3be9eae04162b8fbc140823187e4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/9e4a3be9eae04162b8fbc140823187e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 30 RHS
At line 38 BOUNDS
At line 43 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-4) rows, 4 (0) columns and 9 (-4) elements
0  Obj -0 Dual inf 17.499996 (4)
4  Obj 80.428571
Optimal - objective value 80.428571
After Postsolve, objective 80.428571, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 80.42857143 - 4 iterations time 0.002

In [51]:
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

p1 12.142857
p2 3.5714286
p3 7.4285714
p4 0.0
obj= 80.42857099999999


### ⑧ まとめ

In [52]:
import pandas as pd
import pulp

# データ取得
require_df = pd.read_csv('requires.csv')
stock_df = pd.read_csv('stocks.csv')
gain_df = pd.read_csv('gains.csv')

# リストの定義
P = gain_df['p'].tolist()
M = stock_df['m'].tolist()

# 定数の定義
stock = {row.m:row.stock for row in stock_df.itertuples()}
gain = {row.p:row.gain for row in gain_df.itertuples()}
require = {(row.p, row.m):row.require for row in require_df.itertuples()}

# 数理最適化モデルの定義
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Continuous')

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

# 目的関数の定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

# 計算結果の表示
for p in P:
    print(p, x[p].value())
print('obj=', problem.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/isobekosuke/home/operations_research/venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/218994f1095f48c9827072cde8b6302e-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/218994f1095f48c9827072cde8b6302e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 30 RHS
At line 38 BOUNDS
At line 43 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-4) rows, 4 (0) columns and 9 (-4) elements
0  Obj -0 Dual inf 17.499996 (4)
4  Obj 80.428571
Optimal - objective value 80.428571
After Postsolve, objective 80.428571, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 80.42857143 - 4 iterations time 0.002

### 整数計画問題の場合

In [53]:
import pandas as pd
import pulp

# データ取得
require_df = pd.read_csv('requires.csv')
stock_df = pd.read_csv('stocks.csv')
gain_df = pd.read_csv('gains.csv')

# リストの定義
P = gain_df['p'].tolist()
M = stock_df['m'].tolist()

# 定数の定義
stock = {row.m:row.stock for row in stock_df.itertuples()}
gain = {row.p:row.gain for row in gain_df.itertuples()}
require = {(row.p, row.m):row.require for row in require_df.itertuples()}

# 数理最適化モデルの定義
problem = pulp.LpProblem('IP2', pulp.LpMaximize) # IP: Integer Programming

# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Integer')

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

# 目的関数の定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

# 計算結果の表示
for p in P:
    print(p, x[p].value())
print('obj=', problem.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/isobekosuke/home/operations_research/venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/284f8391f06c4eaaad57df243dc45be7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/b9/s3mpk10s0s1bl06d5p5990h80000gn/T/284f8391f06c4eaaad57df243dc45be7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 38 RHS
At line 46 BOUNDS
At line 51 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 80.4286 - 0.00 seconds
Cgl0004I processed model has 3 rows, 4 columns (4 integer (0 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of -76 found by DiveCoefficient after 0 iterations and 0 